<br><br><p style="text-align:center;font-size:2em;font-weight: bold">Storing the Distance Data Between Addresses</p>

In [ ]:
# Libraries
import numpy as np
import pandas as pd
import urllib
import json 

In [ ]:
# Loader represents the Distance Array between locations 
loader=np.load('distancearray.npy')

In [ ]:
# The csv file contains all of the addresses and corresponding latitude and longitude coordinates
addressdata=pd.read_csv('Myaddrdata.csv')

In [ ]:
# This is the code that updates the rows and columns index of the distance array
# Querying the Google API takes a few hundred milliseconds per query which means 506*505/2=127,765
# Total Time: Around 10 hours
# The code was parallelized to speed up gathering the data
# Still occasionally the API failed to give a bike time so increments of 5 to 10 were performed

for i in range(496,506): # The row indexes to update.
    for j in range(i+1,len(addressdata)): # Only need to find (i,j) for i<j as we just 
                                          # store the (j,i) as the same value 
                                          # (This wouldn't be a good idea for a Hilly City)
            
        # Grab Latitude and Longitude Address Values from the Address Data
        orig_coordlat = addressdata.loc[i][2]
        orig_coordlon = addressdata.loc[i][3]
        dest_coordlat = addressdata.loc[j][2]
        dest_coordlon = addressdata.loc[j][3]
        
        # Code that queries the Google API and stores the bike time 
        # A Google Key is needed to run this code replacing the '...'
        url = "https://maps.googleapis.com/maps/api/distancematrix/json?key=...&origins={0},{1}&destinations={2},{3}&mode=bicycling&language=en-EN&sensor=false".format(str(orig_coordlat),str(orig_coordlon),str(dest_coordlat),str(dest_coordlon))
        json_response = urllib.request.urlopen(url)
        result= json.loads(json_response.read())
        bike_time = result['rows'][0]['elements'][0]['duration']['value']

        # Store the bike time (We assume it's the same in both directions)
        loader[i,j]=bike_time
        loader[j,i]=bike_time

In [ ]:
# Save the Updated distance array to the hard drive
np.save('distancearray.npy',loader)

<br><p style="text-align:center;font-size:2em;font-weight: bold">Function to get latitude and longitude from an address string</p>

In [ ]:
googleGeocodeUrl = 'http://maps.googleapis.com/maps/api/geocode/json?'

def get_coordinates(query, from_sensor=False):
    query = query.encode('utf-8')
    params = {
        'address': query,
        'sensor': "true" if from_sensor else "false"
    }
    url = googleGeocodeUrl + urllib.parse.urlencode(params)
    json_response = urllib.request.urlopen(url)
    response = json.loads(json_response.read())
    if response['results']:
        location = response['results'][0]['geometry']['location']
        latitude, longitude = location['lat'], location['lng']
        print(query, latitude, longitude)
    else:
        latitude, longitude = None, None
        print(query, "<no results>")
    return latitude, longitude

<br>
The data sets were mined to gather all the different data strings that appeared from all the input files (there are around 500 addresses). For most address strings passing it to the above function was sufficient to get the correct latitude and longitude. Some strings however were insufficient or the google api thought the address was for somewhere else (about 40 or so addresses). These addresses were then individually edited to find proper latitude and longitude values.